Business have targets to achieve. We can project sales to predict the business's performance' in future time. It is useful to zoom out and look at the broader picture as well. By considering all our efforts on the customer side, how do we affect the sales?


In [2]:
from __future__ import division

from datetime import datetime, timedelta,date
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

import keras
from keras.layers import Dense, LSTM
from keras.models import Sequential
from keras.optimizers import Adam 
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
from sklearn.model_selection import KFold, cross_val_score, train_test_split

Using TensorFlow backend.
/Users/tridoan/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/tridoan/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/tridoan/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/tridoan/opt/anaconda3/lib/pytho

In [ ]:
import warnings
warnings.filterwarnings("ignore")

We are given 5 years of store-item sales data, and asked to predict 3 months of sales for 50 different items at 10 different stores.

What's the best way to deal with seasonality? Should stores be modeled separately, or can you pool them together? Does deep learning work better than ARIMA? Can either beat xgboost?[data](https://www.kaggle.com/c/demand-forecasting-kernels-only/data)

In [3]:
df_sales = pd.read_csv('./Downloads/data/sale_data.csv')
df_sales.shape

(913000, 4)

In [4]:
df_sales.head(10)

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10
5,2013-01-06,1,1,12
6,2013-01-07,1,1,10
7,2013-01-08,1,1,9
8,2013-01-09,1,1,12
9,2013-01-10,1,1,9


In [5]:
df_sales['date'] = pd.to_datetime(df_sales['date'])

#represent month in date field as its first day
df_sales['date'] = df_sales['date'].dt.year.astype('str') + '-' + df_sales['date'].dt.month.astype('str') + '-01'
df_sales['date'] = pd.to_datetime(df_sales['date'])

df_sales.head()

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-01,1,1,11
2,2013-01-01,1,1,14
3,2013-01-01,1,1,13
4,2013-01-01,1,1,10


Our task is to forecast monthly total sales. We need to aggregate our data at the monthly level and sum up the sales column.

In [6]:
#groupby date and sum the sales
df_sales = df_sales.groupby('date').sales.sum().reset_index()
df_sales.head()

,date,sales
0,2013-01-01,454904
1,2013-02-01,459417
2,2013-03-01,617382
3,2013-04-01,682274
4,2013-05-01,763242


We first deal to work on non-stationary  and trending issue. One method is to get the difference in sales compared to the previous month and build the model on it:

In [7]:
#create a new dataframe to model the difference
df_diff = df_sales.copy()

#add previous sales to the next row
df_diff['prev_sales'] = df_diff['sales'].shift(1)   

df_diff.head()

,date,sales,prev_sales
0,2013-01-01,454904,NaN
1,2013-02-01,459417,454904.0
2,2013-03-01,617382,459417.0
3,2013-04-01,682274,617382.0
4,2013-05-01,763242,682274.0


In [9]:
#drop the null values and calculate the difference
df_diff = df_diff.dropna()

df_diff['diff'] = (df_diff['sales'] - df_diff['prev_sales'])
df_diff.head()

,date,sales,prev_sales,diff
1,2013-02-01,459417,454904.0,4513.0
2,2013-03-01,617382,459417.0,157965.0
3,2013-04-01,682274,617382.0,64892.0
4,2013-05-01,763242,682274.0,80968.0
5,2013-06-01,795597,763242.0,32355.0


We need to use previous monthly sales data to forecast the next ones. The look-back period may vary for every model. Ours will be 12 for this example.
So what we need to do is to create columns from lag_1 to lag_12 and assign values by using shift() method:

In [10]:
#create new dataframe from transformation from time series to supervised
df_supervised = df_diff.drop(['prev_sales'],axis=1)

#adding lags
for inc in range(1,13):
    field_name = 'lag_' + str(inc)
    df_supervised[field_name] = df_supervised['diff'].shift(inc)

df_supervised.head()

,date,sales,diff,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,lag_9,lag_10,lag_11,lag_12
1,2013-02-01,459417,4513.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2013-03-01,617382,157965.0,4513.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2013-04-01,682274,64892.0,157965.0,4513.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2013-05-01,763242,80968.0,64892.0,157965.0,4513.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2013-06-01,795597,32355.0,80968.0,64892.0,157965.0,4513.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
df_supervised.tail(6)

,date,sales,diff,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,lag_9,lag_10,lag_11,lag_12
54,2017-07-01,1171393,106769.0,43938.0,81824.0,116195.0,201298.0,4063.0,-46105.0,-228037.0,27811.0,-33194.0,-84663.0,-157224.0,116054.0
55,2017-08-01,1026403,-144990.0,106769.0,43938.0,81824.0,116195.0,201298.0,4063.0,-46105.0,-228037.0,27811.0,-33194.0,-84663.0,-157224.0
56,2017-09-01,935263,-91140.0,-144990.0,106769.0,43938.0,81824.0,116195.0,201298.0,4063.0,-46105.0,-228037.0,27811.0,-33194.0,-84663.0
57,2017-10-01,891160,-44103.0,-91140.0,-144990.0,106769.0,43938.0,81824.0,116195.0,201298.0,4063.0,-46105.0,-228037.0,27811.0,-33194.0
58,2017-11-01,928837,37677.0,-44103.0,-91140.0,-144990.0,106769.0,43938.0,81824.0,116195.0,201298.0,4063.0,-46105.0,-228037.0,27811.0
59,2017-12-01,695170,-233667.0,37677.0,-44103.0,-91140.0,-144990.0,106769.0,43938.0,81824.0,116195.0,201298.0,4063.0,-46105.0,-228037.0


In [12]:
#drop null values
df_supervised = df_supervised.dropna().reset_index(drop=True)

**How useful are our features for prediction?**
Adjusted R-squared is the answer. It tells us how good our features explain the variation in our label (lag_1 to lag_12 for diff, in our example).

In [13]:
# Import statsmodels.formula.api
import statsmodels.formula.api as smf 

# Define the regression formula
model = smf.ols(formula='diff ~ lag_1', data=df_supervised)

# Fit the regression
model_fit = model.fit()

# Extract the adjusted r-squared
regression_adj_rsq = model_fit.rsquared_adj
print(regression_adj_rsq)

0.02893426930900389


Basically, we fit a linear regression model (OLS — Ordinary Least Squares) and calculate the Adjusted R-squared. For the example above, we just used lag_1 to see how much it explains the variation in column diff. lag_1 explains 3% of the variation.     

Let add more features.

In [14]:


# Define the regression formula
model = smf.ols(formula='diff ~ lag_1 + lag_2 + lag_3 + lag_4 + lag_5', data=df_supervised)

# Fit the regression
model_fit = model.fit()

# Extract the adjusted r-squared
regression_adj_rsq = model_fit.rsquared_adj
print(regression_adj_rsq)

0.4406493613886947


**Adding four more features increased the score from 3% to 44%**. Let use full features


In [16]:
df_supervised[['date','diff','lag_1','lag_2','lag_3','lag_4','lag_5','lag_6','lag_7','lag_8','lag_9','lag_10','lag_11','lag_12']].head()

,date,diff,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,lag_9,lag_10,lag_11,lag_12
0,2014-02-01,3130.0,19380.0,-186036.0,36056.0,-33320.0,-76854.0,-89161.0,60325.0,32355.0,80968.0,64892.0,157965.0,4513.0
1,2014-03-01,175184.0,3130.0,19380.0,-186036.0,36056.0,-33320.0,-76854.0,-89161.0,60325.0,32355.0,80968.0,64892.0,157965.0
2,2014-04-01,84613.0,175184.0,3130.0,19380.0,-186036.0,36056.0,-33320.0,-76854.0,-89161.0,60325.0,32355.0,80968.0,64892.0
3,2014-05-01,93963.0,84613.0,175184.0,3130.0,19380.0,-186036.0,36056.0,-33320.0,-76854.0,-89161.0,60325.0,32355.0,80968.0
4,2014-06-01,23965.0,93963.0,84613.0,175184.0,3130.0,19380.0,-186036.0,36056.0,-33320.0,-76854.0,-89161.0,60325.0,32355.0


In [18]:


# Define the regression formula
model = smf.ols(formula='diff ~ lag_1 + lag_2 + lag_3 + lag_4 + lag_5 + lag_6 + lag_7 + lag_8 + lag_9 + lag_10 + lag_11 + lag_12', data=df_supervised)

# Fit the regression
model_fit = model.fit()

# Extract the adjusted r-squared
regression_adj_rsq = model_fit.rsquared_adj
print(regression_adj_rsq)

0.9795722233296558


The result is impressive as the score is 98%. Now we can confidently build our model after scaling our data. But there is one more step before scaling. We should split our data into train and test sets. As the test set, we have selected the last 6 months’ sales.

In [19]:
df_supervised.shape

(47, 15)

In [20]:
#import MinMaxScaler and create a new dataframe for LSTM model

from sklearn.preprocessing import MinMaxScaler

df_model = df_supervised.drop(['sales','date'],axis=1)

#split train and test set
train_set, test_set = df_model[0:-6].values, df_model[-6:].values

df_model.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47 entries, 0 to 46
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   diff    47 non-null     float64
 1   lag_1   47 non-null     float64
 2   lag_2   47 non-null     float64
 3   lag_3   47 non-null     float64
 4   lag_4   47 non-null     float64
 5   lag_5   47 non-null     float64
 6   lag_6   47 non-null     float64
 7   lag_7   47 non-null     float64
 8   lag_8   47 non-null     float64
 9   lag_9   47 non-null     float64
 10  lag_10  47 non-null     float64
 11  lag_11  47 non-null     float64
 12  lag_12  47 non-null     float64
dtypes: float64(13)
memory usage: 4.9 KB


In [22]:
print(' train ',train_set.shape)

 train  (41, 13)


In [23]:
#apply Min Max Scaler
scaler = MinMaxScaler(feature_range=(-1, 1))
scaler = scaler.fit(train_set)

# reshape training set
train_set = train_set.reshape(train_set.shape[0], train_set.shape[1])

train_set_scaled = scaler.transform(train_set)

# reshape test set
test_set = test_set.reshape(test_set.shape[0], test_set.shape[1])
test_set_scaled = scaler.transform(test_set)

In [25]:
print(' train shape ',train_set_scaled.shape, ' test shape ', test_set.shape)

 train shape  (41, 13)  test shape  (6, 13)


In [26]:
X_train, y_train = train_set_scaled[:, 1:], train_set_scaled[:, 0:1]
X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])

X_test, y_test = test_set_scaled[:, 1:], test_set_scaled[:, 0:1]
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

In [29]:
X_train[:3]

array([[[ 0.15255919, -0.80434393,  0.23024212, -0.0447346 ,
         -0.25830878, -0.3186859 ,  0.40696724,  0.26794062,
          0.50957454,  0.42966779,  0.8922929 ,  0.12955024]],

       [[ 0.07686073,  0.15255919, -0.80434393,  0.29561828,
         -0.0447346 , -0.25830878, -0.33606217,  0.40696724,
          0.26794062,  0.50957454,  0.42966779,  0.8922929 ]],

       [[ 0.8783514 ,  0.07686073,  0.15255919, -0.79394659,
          0.29561828, -0.0447346 , -0.27488947, -0.33606217,
          0.40696724,  0.26794062,  0.50957454,  0.42966779]]])

### Building the LSTM model   
Everything is ready to build our first deep learning model.

In [30]:
model = Sequential()
model.add(LSTM(4, batch_input_shape=(1, X_train.shape[1], X_train.shape[2]), stateful=True))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, y_train, nb_epoch=100, batch_size=1, verbose=1, shuffle=False)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


/Users/tridoan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """


Epoch 1/100
41/41 [==============================] - 0s 7ms/step - loss: 0.4305
Epoch 2/100
41/41 [==============================] - 0s 879us/step - loss: 0.3638
Epoch 3/100
41/41 [==============================] - 0s 863us/step - loss: 0.3158
Epoch 4/100
41/41 [==============================] - 0s 826us/step - loss: 0.2754
Epoch 5/100
41/41 [==============================] - 0s 848us/step - loss: 0.2406
Epoch 6/100
41/41 [==============================] - 0s 841us/step - loss: 0.2102
Epoch 7/100
41/41 [==============================] - 0s 869us/step - loss: 0.1834
Epoch 8/100
41/41 [==============================] - 0s 888us/step - loss: 0.1595
Epoch 9/100
41/41 [==============================] - 0s 853us/step - loss: 0.1383
Epoch 10/100
41/41 [==============================] - 0s 874us/step - loss: 0.1196
Epoch 11/100
41/41 [==============================] - 0s 866us/step - loss: 0.1033
Epoch 12/100
41/41 [==============================] - 0s 879us/step - loss: 0.0894
Epoch 13/100
41

Let do prediction.

In [31]:
y_pred = model.predict(X_test,batch_size=1)
y_pred

array([[ 0.66558945],
       [-0.5835324 ],
       [-0.39540625],
       [-0.03602177],
       [ 0.19803987],
       [-1.0476856 ]], dtype=float32)

In [32]:
len(y_pred)

6

hese are scaled data that shows the difference. How we can see the actual sales prediction?   
First, we need to do the **inverse transformation for scaling**:

In [34]:
#reshape y_pred
y_pred = y_pred.reshape(y_pred.shape[0], 1, y_pred.shape[1])

y_pred[:3]

array([[[ 0.66558945]],

       [[-0.5835324 ]],

       [[-0.39540625]]], dtype=float32)

In [35]:
#rebuild test set for inverse transform
pred_test_set = []

for index in range(0,len(y_pred)):
    print(np.concatenate([y_pred[index],X_test[index]],axis=1))
    pred_test_set.append(np.concatenate([y_pred[index],X_test[index]],axis=1))

pred_test_set[0]

[[ 0.66558945  0.26695937  0.44344626  0.60355899  1.10628178  0.13866328
  -0.10745675 -1.02635392  0.24535439 -0.05787474 -0.31370458 -0.67437352
   0.68397168]]
[[-0.58353239  0.55964922  0.26695937  0.44344626  0.68877355  1.10628178
   0.13866328 -0.12204966 -1.02635392  0.24535439 -0.05787474 -0.31370458
  -0.67437352]]
[[-0.39540625 -0.61313659  0.55964922  0.26695937  0.52015228  0.68877355
   1.10628178  0.12731349 -0.12204966 -1.02635392  0.24535439 -0.05787474
  -0.31370458]]
[[-0.03602177 -0.36228353 -0.61313659  0.55964922  0.33428672  0.52015228
   0.68877355  1.10768225  0.12731349 -0.12204966 -1.02635392  0.24535439
  -0.05787474]]
[[ 0.19803987 -0.14316792 -0.36228353 -0.61313659  0.64253037  0.33428672
   0.52015228  0.68467253  1.10768225  0.12731349 -0.12204966 -1.02635392
   0.24535439]]
[[-1.04768562  0.23779333 -0.14316792 -0.36228353 -0.59257833  0.64253037
   0.33428672  0.51382935  0.68467253  1.10768225  0.12731349 -0.12204966
  -1.02635392]]


array([[ 0.66558945,  0.26695937,  0.44344626,  0.60355899,  1.10628178,
         0.13866328, -0.10745675, -1.02635392,  0.24535439, -0.05787474,
        -0.31370458, -0.67437352,  0.68397168]])

In [36]:
len(pred_test_set)

6

In [37]:
#reshape pred_test_set
pred_test_set = np.array(pred_test_set)
pred_test_set.shape

(6, 1, 13)

In [38]:

pred_test_set = pred_test_set.reshape(pred_test_set.shape[0], pred_test_set.shape[2])

#inverse transform
pred_test_set_inverted = scaler.inverse_transform(pred_test_set)


In [39]:
#create dataframe that shows the predicted sales
result_list = []
sales_dates = list(df_sales[-7:].date)
act_sales = list(df_sales[-7:].sales)
for index in range(0,len(pred_test_set_inverted)):
    result_dict = {}
    result_dict['pred_value'] = int(pred_test_set_inverted[index][0] + act_sales[index])
    result_dict['date'] = sales_dates[index+1]
    result_list.append(result_dict)
df_result = pd.DataFrame(result_list)

In [40]:
df_result.head()

,pred_value,date
0,1194134,2017-07-01
1,1032758,2017-08-01
2,928152,2017-09-01
3,914160,2017-10-01
4,920303,2017-11-01


In [41]:
df_sales.head()

,date,sales
0,2013-01-01,454904
1,2013-02-01,459417
2,2013-03-01,617382
3,2013-04-01,682274
4,2013-05-01,763242


In [42]:
#merge with actual sales dataframe
df_sales_pred = pd.merge(df_sales,df_result,on='date',how='left')

df_sales_pred

,date,sales,pred_value
0,2013-01-01,454904,NaN
1,2013-02-01,459417,NaN
2,2013-03-01,617382,NaN
3,2013-04-01,682274,NaN
4,2013-05-01,763242,NaN
5,2013-06-01,795597,NaN
6,2013-07-01,855922,NaN
7,2013-08-01,766761,NaN
8,2013-09-01,689907,NaN
9,2013-10-01,656587,NaN
